# EU Cities Cost of Living Comparison

In [30]:
import os
import warnings
import json
import pandas as pd

In [2]:
os.chdir("..")
warnings.filterwarnings("ignore")

In [3]:
from scraper.scrape import API, URL

In [4]:
def get_city_info(country: str, city: str):
    api = API(URL, country)
    return api.get_single_city(city)

In [5]:
def format_keys(city_info: dict):
    return {elem[2:]: city_info[elem] for elem in list(city_info.keys())}

In [23]:
def get_formatted_price(
    city_info: dict, 
    group_key: str = "", 
    price_idx: tuple = (0, 1)):
    
    price_str = city_info[group_key][price_idx[0]][price_idx[1]]
    price_str = price_str.replace(u'\xa0', u'').strip()[:-1]
    price_f = float(price_str.replace(',', ''))
    return round(price_f, 2)

In [7]:
EXPENSES_FEATURES = ('Restaurants', 'Markets', 'Utilities (Monthly)')
RENT_FEATURE = 'Rent Per Month'
TOTAL_FEATURE = 'Cost of Living'

In [25]:
def summary_expenses(city_info: dict):
    expenses = {f: round(sum([get_formatted_price(city_info, f, (i, 1)) for i in range(len(city_info[f]))]), 2)
                    for f in EXPENSES_FEATURES}
    expenses[RENT_FEATURE] = get_formatted_price(city_info, RENT_FEATURE, (0, 1))
    expenses[TOTAL_FEATURE] = round(sum(expenses.values()), 2)
    return expenses

In [9]:
def get_city_summary(country: str, city: str):
    city_info = format_keys(get_city_info(country, city))
    return summary_expenses(city_info)

### Examples: single city

In [27]:
get_city_summary("Netherlands", "Amsterdam")

{'Restaurants': 112.92,
 'Markets': 74.4,
 'Utilities (Monthly)': 215.34,
 'Rent Per Month': 1536.9,
 'Cost of Living': 1939.56}

In [28]:
get_city_summary("France", "Paris")

{'Restaurants': 105.62,
 'Markets': 97.94,
 'Utilities (Monthly)': 213.01,
 'Rent Per Month': 1292.37,
 'Cost of Living': 1708.94}

In [29]:
get_city_summary("United Kingdom", "London")

{'Restaurants': 96.6,
 'Markets': 64.31,
 'Utilities (Monthly)': 227.11,
 'Rent Per Month': 1724.53,
 'Cost of Living': 2112.55}

## Getting multiple

In [35]:
cities = [
    ("Netherlands", "Amsterdam"),
    ("France", "Paris"),
    ("United Kingdom", "London"),
    ("Portugal", "Lisbon"),
    ("Spain", "Barcelona"),
    ("Italy", "Rome"),
    ("Germany", "Berlin"),
    ("Austria", "Vienna"),
]

In [42]:
info = {c[1]: get_city_summary(c[0], c[1]) for c in cities}

In [43]:
pd.DataFrame(info)

,Amsterdam,Paris,London,Lisbon,Barcelona,Rome,Berlin,Vienna
Restaurants,112.92,105.62,96.60,65.37,81.54,95.15,82.60,87.04
Markets,74.40,97.94,64.31,47.23,57.69,73.56,64.45,70.92
Utilities (Monthly),215.34,213.01,227.11,162.85,196.67,225.09,305.20,224.13
Rent Per Month,1536.90,1292.37,1724.53,865.17,904.36,992.57,1048.38,874.62
Cost of Living,1939.56,1708.94,2112.55,1140.62,1240.26,1386.37,1500.63,1256.71
